# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
# Your code here
import sqlite3
import pandas as pd

conn = sqlite3.connect('data.sqlite')
cursor = conn.cursor()

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [2]:
# Your code here
query = '''
SELECT firstName, lastName
FROM employees AS e 
JOIN offices AS o
ON e.officeCode = o.officecode
WHERE o.city = 'Boston';
'''

pd.read_sql(query, conn)

,firstName,lastName
0,Julie,Firrelli
1,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [3]:
# Your code here
query1 = '''
SELECT o.officeCode, o.city, COUNT(e.employeeNumber) AS number_of_employees
FROM offices AS o
LEFT JOIN employees AS e
ON o.officeCode = e.officecode
GROUP BY o.officeCode, o.city
HAVING COUNT(e.employeeNumber) = 0;
'''

pd.read_sql(query1, conn)

,officeCode,city,number_of_employees
0,27,Boston,0


## Write 3 questions of your own and answer them

In [4]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""

'\nHow many customers are there per office?\n'

In [5]:
"""
Question 1
How many orders has each customer placed?
"""

# Your code here
query2 = '''
SELECT c.customerName, COUNT(o.orderNumber) AS orderCount
FROM customers c
LEFT JOIN orders o ON c.customerNumber = o.customerNumber
GROUP BY c.customerName;
'''

pd.read_sql(query2, conn)

,customerName,orderCount
0,ANG Resellers,0
1,"AV Stores, Co.",3
2,Alpha Cognac,3
3,American Souvenirs Inc,0
4,Amica Models & Co.,2
...,...,...
117,Vitachrome Inc.,3
118,"Volvo Model Replicas, Co",4
119,Warburg Exchange,0
120,West Coast Collectables Co.,2


In [6]:
"""
Question 2
What is the total sales amount for each product?
"""

# Your code here
query3 = '''
SELECT  p.productName, SUM(od.quantityOrdered * od.priceEach) AS total_sales
FROM products AS p
JOIN orderdetails AS od 
ON p.productCode = od.productCode
GROUP BY p.productName
ORDER BY total_sales DESC;
'''
pd.read_sql(query3, conn)

,productName,total_sales
0,1992 Ferrari 360 Spider red,276839.98
1,2001 Ferrari Enzo,190755.86
2,1952 Alpine Renault 1300,190017.96
3,2003 Harley-Davidson Eagle Drag Bike,170686.00
4,1968 Ford Mustang,161531.48
...,...,...
104,1982 Ducati 996 R,33268.76
105,1958 Chevy Corvette Limited Edition,31627.96
106,1982 Lamborghini Diablo,30972.87
107,1936 Mercedes Benz 500k Roadster,29763.39


In [7]:
"""
Question 3
How many customers does each employee manage?
"""

# Your code here
query4 = '''
SELECT e.firstName, e.lastName, COUNT(c.customerNumber) AS number_of_customers
FROM employees AS e
LEFT JOIN customers AS c 
ON e.employeeNumber = c.salesRepEmployeeNumber
GROUP BY e.firstName, e.lastName
ORDER BY number_of_customers DESC;
'''
pd.read_sql(query4, conn)

,firstName,lastName,number_of_customers
0,Pamela,Castillo,10
1,Barry,Jones,9
2,George,Vanauf,8
3,Larry,Bott,8
4,Foon Yue,Tseng,7
5,Gerard,Hernandez,7
6,Julie,Firrelli,6
7,Leslie,Jennings,6
8,Leslie,Thompson,6
9,Loui,Bondur,6


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [8]:
# Your code here
query5 = '''
SELECT e.firstName, e.lastName, p.productName
FROM employees AS e
JOIN customers AS c 
ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders AS o 
ON c.customerNumber = o.customerNumber
JOIN orderdetails AS od 
ON o.orderNumber = od.orderNumber
JOIN products AS p 
ON od.productCode = p.productCode
ORDER BY e.firstName, e.lastName, p.productName;
'''
pd.read_sql(query5, conn)

,firstName,lastName,productName
0,Andy,Fixter,18th Century Vintage Horse Carriage
1,Andy,Fixter,18th Century Vintage Horse Carriage
2,Andy,Fixter,18th Century Vintage Horse Carriage
3,Andy,Fixter,18th Century Vintage Horse Carriage
4,Andy,Fixter,1900s Vintage Bi-Plane
...,...,...,...
2991,Steve,Patterson,The Schooner Bluenose
2992,Steve,Patterson,The Titanic
2993,Steve,Patterson,The Titanic
2994,Steve,Patterson,The USS Constitution Ship


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [9]:
# Your code here
query6 = '''
SELECT e.firstName, e.lastName, SUM(od.quantityOrdered) AS quantity_sold
FROM employees AS e
JOIN customers AS c 
ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders AS o 
ON c.customerNumber = o.customerNumber
JOIN orderdetails AS od 
ON o.orderNumber = od.orderNumber
GROUP BY e.firstName, e.lastName
ORDER BY quantity_sold;
'''
pd.read_sql(query6, conn)

,firstName,lastName,quantity_sold
0,Leslie,Thompson,4056
1,Martin,Gerard,4180
2,Julie,Firrelli,4227
3,Mami,Nishi,4923
4,Foon Yue,Tseng,5016
5,Steve,Patterson,5561
6,Loui,Bondur,6186
7,Andy,Fixter,6246
8,Peter,Marsh,6632
9,George,Vanauf,7423


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [10]:
# Your code here
query7 = '''
SELECT e.firstName, e.lastName, COUNT(DISTINCT p.productName) AS products_sold
FROM employees AS e
JOIN customers AS c 
ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders AS o 
ON c.customerNumber = o.customerNumber
JOIN orderdetails AS od 
ON o.orderNumber = od.orderNumber
JOIN products AS p
ON od.productCode = p.productCode
GROUP BY e.firstName, e.lastName
HAVING products_sold > 200
ORDER BY products_sold;
'''
pd.read_sql(query7, conn)

,firstName,lastName,products_sold


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!